In [ ]:
!pip install tqdm

In [ ]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
ytt = load_dataset(
    "pinecone/yt-transcriptions",
    split="train",
    revision="926a45"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
#import pinecone  # pip install pinecone-client

# connect to pinecone (get API key and env at app.pinecone.io)
from pinecone import Pinecone, PodSpec

pc = Pinecone(api_key="b548349d-858c-44cd-8e13-b56ad80eab3e")
# create index
pc.create_index(

	'youtube-search',
  	dimension=768, metric='cosine',
	spec=PodSpec(
    environment="gcp-starter"
  )
)
# connect to the new index
index = pc.Index('youtube-search')

In [15]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a075a42df0bbdeb707904ecea8723411fd0dbd6153cbfb349a1870cbf9dede91
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [16]:
from sentence_transformers import SentenceTransformer

retriever = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')
retriever

.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [17]:
from tqdm.auto import tqdm

docs = []  # this will store IDs, embeddings, and metadata

batch_size = 32

for i in tqdm(range(0, len(ytt), batch_size)):
    i_end = min(i+batch_size, len(ytt))
    # extract batch from YT transactions data
    batch = ytt[i:i_end]
    # encode batch of text
    embeds = retriever.encode(batch['text']).tolist()
    # each snippet needs a unique ID
    # we will merge video ID and start_seconds for this
    ids = [f"{x[0]}-{x[1]}" for x in zip(batch['video_id'], batch['start_second'])]
    # create metadata records
    meta = [{
        'video_id': x[0],
        'title': x[1],
        'text': x[2],
        'start_second': x[3],
        'end_second': x[4],
        'url': x[5],
        'thumbnail': x[6]
    } for x in zip(
        batch['video_id'],
        batch['title'],
        batch['text'],
        batch['start_second'],
        batch['end_second'],
        batch['url'],
        batch['thumbnail']
    )]
    # create list of (IDs, vectors, metadata) to upsert
    to_upsert = list(zip(ids, embeds, meta))
    # add to pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/354 [00:00<?, ?it/s]

In [18]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.11298,
 'namespaces': {'': {'vector_count': 11298}},
 'total_vector_count': 11298}

In [19]:
query = "What is deep learning?"

xq = retriever.encode([query]).tolist()

In [ ]:
xq

In [26]:
xc['matches']

[{'id': 'Z6rxFNMGdn0-1154',
  'metadata': {'end_second': 1181.0,
               'start_second': 1154.0,
               'text': " terms of optimization but what's the algorithm for "
                       'updating the parameters or updating whatever the state '
                       'of the network is and then the the last part is the the '
                       'data set like how do you actually represent the world '
                       'as it comes into your machine learning system so I '
                       'think of deep learning as telling us something about '
                       'what does the model look like and basically to qualify '
                       'as deep I',
               'thumbnail': 'https://i.ytimg.com/vi/Z6rxFNMGdn0/maxresdefault.jpg',
               'title': 'Ian Goodfellow: Generative Adversarial Networks (GANs) '
                        '| Lex Fridman Podcast #19',
               'url': 'https://www.youtube.com/watch?v=Z6rxFNMGdn0&t=1154s',
      

In [ ]:
xc = index.query(vector = xq, top_k=3,
              include_metadata=True)

In [27]:
for context in xc['matches']:
    print(context['metadata']['text'], end="\n---\n")

 terms of optimization but what's the algorithm for updating the parameters or updating whatever the state of the network is and then the the last part is the the data set like how do you actually represent the world as it comes into your machine learning system so I think of deep learning as telling us something about what does the model look like and basically to qualify as deep I
---
 any theoretical components any theoretical things that you need to understand about deep learning can be sick later for that link again just watched the word doc file again in that I mentioned the link also the second channel is my channel because deep learning might be complete deep learning playlist that I have created is completely in order okay to the other
---
 under a rock for the last few years you have heard of the deep networks and how they have revolutionised computer vision and kind of the standard classic way of doing this is it's basically a classic supervised learning problem you are givi